# GridSearchCV - Lab

## Introduction

In this lab, we'll explore how to use scikit-learn's `GridSearchCV` class to exhaustively search through every combination hyperparameters until we find the values for a given model.

## Objectives

You will be able to:

* Understand and explain parameter tuning and why it is necessary 
* Design and create a parameter grid for use with sklearn's GridSearchCV module
* Use GridSearchCV to increase model performance through parameter tuning


## The Dataset

For this lab, we'll be working with the [Wine Quality Dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality) from the UCI Machine Learning Dataset Repository.  We'll be using data about the various features of wine to predict the quality of the wine on a scale from 1-10 stars, making this a multiclass classification problem.  

### Getting Started

Before we can begin GridSearching our way to optimal hyperparameters, we'll need to go through the basic steps of modeling.  This means that we'll need to:

* Import and inspect the dataset (and clean, if necessary)
* Split the data into training and testing sets
* Build and fit a baseline model that we can compare against our GridSearch results.

Run the cell below to import everything we'll need for this lab.  

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score

/opt/conda/envs/learn-env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/opt/conda/envs/learn-env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/opt/conda/envs/learn-env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/envs/learn-env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/opt/conda/envs/learn-env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from

Now that we've imported all the necessary libraries and frameworks for this lab, we'll need to get the dataset.  

Our data is stored in the file `winequality-red.csv`. Use pandas to import the data from this file and store it in a DataFrame.  Print the head to ensure that everything loaded correctly. 

In [5]:
df = pd.read_csv('winequality-red.csv')

Great! Let's inspect our data a bit.  In the cell below, perform some basic Exploratory Data Analysis on our dataset.  Get a feel for your data by exploring the descriptive statistics and creating at least 1 visualization to help you better understand this dataset.

In [6]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [7]:
dest= df.describe().T

In [8]:
dest['maxZ']=(dest['max']-dest['mean'])/(.00001+dest['std'])

In [9]:
dest['minZ']=-1*(dest['min']-dest['mean'])/(.00001+dest['std'])

In [10]:
dest.sort_values(by='maxZ')

,count,mean,std,min,25%,50%,75%,max,maxZ,minZ
quality,1599.0,5.636023,0.807569,3.00000,5.0000,6.00000,6.000000,8.00000,2.927238,3.264103
density,1599.0,0.996747,0.001887,0.99007,0.9956,0.99675,0.997835,1.00369,3.659514,3.518979
citric acid,1599.0,0.270976,0.194801,0.00000,0.0900,0.26000,0.420000,1.00000,3.742211,1.390966
alcohol,1599.0,10.422983,1.065668,8.40000,9.5000,10.20000,11.100000,14.90000,4.201099,1.898307
fixed acidity,1599.0,8.319637,1.741096,4.60000,7.1000,7.90000,9.200000,15.90000,4.353762,2.136364
pH,1599.0,3.311113,0.154386,2.74000,3.2100,3.31000,3.400000,4.01000,4.526573,3.699004
free sulfur dioxide,1599.0,15.874922,10.460157,1.00000,7.0000,14.00000,21.000000,72.00000,5.365601,1.422054
volatile acidity,1599.0,0.527821,0.179060,0.12000,0.3900,0.52000,0.640000,1.58000,5.875810,2.277440
total sulfur dioxide,1599.0,46.467792,32.895324,6.00000,22.0000,38.00000,62.000000,289.00000,7.372845,1.230199
sulphates,1599.0,0.658149,0.169507,0.33000,0.5500,0.62000,0.730000,2.00000,7.915733,1.935787


In [11]:
df['quality']=df.quality.astype(str)

In [12]:
for col in df.describe():
    df=df[abs(df[col]-df[col].mean())<4*df[col].std()]

In [13]:
dict(df.nunique())

{'fixed acidity': 93,
 'volatile acidity': 141,
 'citric acid': 78,
 'residual sugar': 77,
 'chlorides': 129,
 'free sulfur dioxide': 54,
 'total sulfur dioxide': 140,
 'density': 425,
 'pH': 83,
 'sulphates': 82,
 'alcohol': 63,
 'quality': 6}

In [14]:
counts=0
for col in df.describe().columns:
    counts+= sum(df[col]<0)

In [15]:
counts

0

In [16]:
len(df)

1536

**_Question:_** Based on your findings during your Eploratory Data Analysis, do you think that we need to do any sort of preprocessing on this dataset? Why or why not?

Write your answer below this line:
________________________________________________________________________________________________________________________________


### Preprocessing our Data

Now, we'll perform any necessary preprocessing on our dataset before training our model.  We'll start by isolating the target variable that we are trying to predict.  In the cell below:

* Store the data in the `quality` column inside the `labels` variable
* Drop the `quality` column from the dataset

In [17]:
df.quality= df.quality.astype(str)
qual=df.quality
df.drop(columns='quality', inplace=True)

Now that we've isolated our labels, we'll need to normalize our dataset (also referred to as _scaling_).  

In the cell below:

* Create a `StandardScaler()` object.
* Transform the data in `labels_removed_df` using the scaler object's `fit_transform()` method.

In [18]:
scaler = StandardScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(df))
scaled_df.columns=df.columns

In [19]:
scaled_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,-0.529989,0.984095,-1.388627,-0.549140,-0.259949,-0.461994,-0.365898,0.600768,1.312192,-0.630425,-0.977603
1,-0.295016,2.010763,-1.388627,0.180378,0.618822,0.944194,0.690168,0.054049,-0.793897,0.248147,-0.600116
2,-0.295016,1.326318,-1.179429,-0.132273,0.379157,-0.060226,0.274142,0.163393,-0.386267,0.028504,-0.600116
3,1.702252,-1.411465,1.540145,-0.549140,-0.299893,0.140658,0.466154,0.710112,-1.065650,-0.483996,-0.600116
4,-0.529989,0.984095,-1.388627,-0.549140,-0.259949,-0.461994,-0.365898,0.600768,1.312192,-0.630425,-0.977603


### Training, Testing, and Cross Validation

Normally, we would also split our data into training and testing sets.  However, since we'll be making use of **_Cross Validation_** when using `GridSearchCV`, we'll also want to make use of it with our baseline model to ensure that things are equal.  Recall that we do not need to split our data into training and testing sets when using cross validation, since the cross validation will take care of that for us.  

### Creating a Baseline Model: Decision Trees

In the cell below:
* Create a `DecisionTreeClassifier` object.  
* Get the `cross_val_score` for this model, with the `cv` parameter set to `3`.
* Calculate and print the mean cross-validation score from our model.

**_Note:_** If you need a refresher on how to use `cross_val_score`, check out the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html).

In [20]:
dt_clf = DecisionTreeClassifier()
dt_cv_score = cross_val_score(dt_clf,scaled_df, qual, cv=3)
mean_dt_cv_score = np.mean(dt_cv_score)

# print("Mean Cross Validation Score: {:.4}%".format(mean_dt_cv_score * 100))

## Grid Search: Decision Trees

Take a second to interpret the results of our cross-validation score.  How well did our model do? How does this compare to a naive baseline level of accuracy (random guessing)?

Write your answer below this line:
________________________________________________________________________________________________________________________________



### Creating A Parameter Grid

So far, our model has not have stellar performance. However, we've yet to modify the hyperparameters of the model.  Each dataset is different, and the chances that the best possible parameters for a given dataset also happen to be the default parameters set by by sklearn at instantiation is very low.  

This means that we need to try **_Hyperparameter Tuning_**.  There are several strategies for searching for optimal hyperparameters--the one we'll be using, **_Combinatoric Grid Searching_**, is probably the most popular, because it performs an exhaustive search of all possible combinations.  

The sklearn module we'll be using to accomplish this is `GridSearchCV`, which can be found inside of `sklearn.model_selection`.

Take a minute to look at sklearn's user guide for [GridSearchCV](http://scikit-learn.org/stable/modules/grid_search.html#grid-search), and then complete the following task.

In the cell below:

* Complete the `param_grid` dictionary.  In this dictionary, each key represents a parameter we want to tune, whereas the corresponding value is an array of every parameter value we'd like to check for that parameter.  For instance, if we would like try out the values `2`, `5`, and `10` for `min_samples_split`, our `param_grid` dictionary would include `"min_samples_split": [2, 5, 10]`.
* Normally, you would have to just try different values to search through for each parameter.  However, in order to limit the complexity of this lab, the parameters and values to search through have been provided for you.  You just need to turn them into key-value pairs inside of the `param_grid` dictionary. Complete `param_grid` so that it tests the following values for each corresponding parameter:
    * For `"criterion"`, try values of `"gini"` and `"entropy"`.
    * For `"max_depth"`, try `None`, as well as `2, 3, 4, 5` and `6`.
    * For `min_samples_split`, try `2, 5`, and `10`.
    * For `"min_samples_leaf"`, try `1, 2, 3, 4, 5` and `6`.

In [31]:
dt_param_grid = {'criterion': ['gini', 'entropy'],
                 'max_depth': [None, 2,3,4,5,6],
                 'min_samples_split': [2,5,10],
                 'min_samples_leaf': range(1,7)
 
}

Now that we have our parameter grid set up, we can create and use our `GridSearchCV` object.  Before we do, let's briefly think about the particulars of this model. 

Grid Searching works by training a model on the data for each unique combination of parameters, and then returning the parameters of the model that performed best. In order to protect us from randomness, it is common to implement K-Fold Cross Validation during this step.  For this lab, we'll set K = 3, meaning that we'll actually train 3 different models for each unique combination of parameters.  

Given our `param_grid` and the knowledge that we're going to use Cross Validation with a value of 3, how many different Decision Trees will our `GridSearchCV` object have to train in order to try every possible combination and find the best parameter choices?

Calculate and print your answer in the cell below.

In [32]:
num_decision_trees = 3*6*3*6*2
print("Grid Search will have to search through {} different permutations.".format(num_decision_trees))

Grid Search will have to search through 648 different permutations.


That's alot of Decision Trees! Decision Trees are generally pretty quick to train, but that isn't the case with every type of model we could want to tune.  Be aware that if you set a particularly large search space of parameters inside your parameter grid, then Grid Searching could potentially take a very long time. 

Let's create our `GridSearchCV` object and fit it.  In the cell below:
* Create a `GridSearchCV` object.  Pass in our model, the parameter grid, and `cv=3` to tell the object to use 3-Fold Cross Validation. Also pass in `return`
* Call our grid search object's `fit()` method and pass in our data and labels, just as if we were using regular cross validation.  
* 

In [33]:
dt_grid_search = GridSearchCV(dt_clf, dt_param_grid,
                              cv=3, return_train_score=True)
#dt_grid_search.fit(None, None)

In [34]:
dt_grid_search.fit(scaled_df, pd.DataFrame(qual))

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [None, 2, 3, 4, 5, 6], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': range(1, 7)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

### Examining the Best Parameters

Now that we have fit our model using Grid Search, we need to inspect it to discover the optimal combination of parameters.

In the cell below:

* Calculate the the mean training score.  An array of training score results can be found inside of the `.cv_results_` dictionary, with the key `mean_train_score`.
* Calcuate the testing score using the our grid search model's `.score()` method by passing in our data and labels. 
* Examine the appropriate attribute to discover the best estimator parameters found during the grid search. 

**_HINT:_** If you're unsure what attribute this is stored in, take a look at sklearn's [GridSearchCV Documentation](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

In [36]:
pd.DataFrame(dt_grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.011348,0.000868,0.001441,0.000032,gini,None,1,2,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.423002,0.447266,0.467710,0.445964,0.018275,216,1.000000,1.000000,1.000000,1.000000,0.000000
1,0.018214,0.011212,0.001540,0.000121,gini,None,1,5,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.452242,0.453125,0.495108,0.466797,0.019993,211,0.951124,0.945312,0.947317,0.947918,0.002410
2,0.009939,0.000658,0.001446,0.000075,gini,None,1,10,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.450292,0.462891,0.514677,0.475911,0.027851,207,0.889541,0.885742,0.886829,0.887371,0.001597
3,0.009601,0.000510,0.001430,0.000073,gini,None,2,2,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.409357,0.449219,0.502935,0.453776,0.038339,215,0.925709,0.917969,0.924878,0.922852,0.003469
4,0.009146,0.000402,0.001365,0.000032,gini,None,2,5,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.446394,0.435547,0.497065,0.459635,0.026796,214,0.914956,0.906250,0.910244,0.910483,0.003558
5,0.010250,0.000385,0.001715,0.000471,gini,None,2,10,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.434698,0.478516,0.520548,0.477865,0.035051,205,0.867058,0.860352,0.865366,0.864258,0.002848
6,0.009402,0.000950,0.001248,0.000026,gini,None,3,2,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.463938,0.476562,0.516634,0.485677,0.022458,193,0.881720,0.879883,0.880976,0.880860,0.000755
7,0.009060,0.000830,0.001330,0.000031,gini,None,3,5,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.458090,0.470703,0.526419,0.485026,0.029677,197,0.882698,0.878906,0.875122,0.878909,0.003093
8,0.009133,0.000302,0.001348,0.000077,gini,None,3,10,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.442495,0.484375,0.520548,0.482422,0.031895,200,0.847507,0.839844,0.849756,0.845702,0.004243
9,0.009126,0.000541,0.001360,0.000087,gini,None,4,2,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.438596,0.470703,0.528376,0.479167,0.037138,203,0.848485,0.851562,0.840976,0.847008,0.004447


In [9]:
dt_gs_training_score = 
dt_gs_testing_score = None

# print("Mean Training Score: {:.4}%".format(dt_gs_training_score * 100))
# print("Mean Testing Score: {:.4}%".format(dt_gs_testing_score * 100))
# print("Best Parameter Combination Found During Grid Search:")
# dt_grid_search.best_params_

**_Question:_** What effect, if any, did our parameter tuning have on model performance? Will GridSearchCV always discover a perfectly (global) optimal set of parameters? Why or why not?
________________________________________________________________________________________________________________________________
  

### Tuning More Advanced Models: Random Forests

Now that we have some experience with Grid Searching through parameter values for a Decision Tree Classifier, let's try our luck with a more advanced model and tune a _Random Forest Classifier_.  

We'll start by repeating the same process we did for our Decision Tree Classifier, except with a Random Forest Classifier instead. 

In the cell below:
* Create a `RandomForestClassifier` object.
* Use Cross Validation with `cv=3` to generate a baseline score for this model type, so that we have something to compare our tuned model performance to. 

In [10]:
rf_clf = None
mean_rf_cv_score = None
# print("Mean Cross Validation Score for Random Forest Classifier: {:.4}%".format(mean_rf_cv_score * 100))

Now that we have our baseline score, we'll create a parameter grid specific to our Random Forest Classifier.  

Again--in a real world situation, you will need to decide what parameters to tune, and be very thoughtful about what values to test for each parameter.  However, since this is a lab, we have provided the following table in the interest of simplicity.  Complete the `rf_param_grid` dictionary with the following key value pairs:
 
 
 |     Parameter     |         Values         |
|:-----------------:|:----------------------:|
|    n_estimators   |      [10, 30, 100]     |
|     criterion     |   ['gini', 'entropy']  |
|     max_depth     | [None, 2, 6, 10] |
| min_samples_split |       [5, 10]       |
|  min_samples_leaf |   [3, 6]   |

In [11]:
rf_param_grid = {
}

Great! Now that we have our parameter grid, we can grid search through it with our Random Forest. 

In the cell below, follow the process we used with Decision Trees above to grid search for the best parameters for our Random Forest Classifier.  

When creating your `GridSearchCV` object,  pass in:
* our Random Forest Classifier
* The parameter grid for our Random Forest Classifier
* `cv=3` 
* **_Do not_** pass in `return_train_score` as we did with our Decision Trees example above.  In the interest of runtime, we'll only worry about testing accuracy this time. 


**_NOTE:_** The runtime on the following cell will be over a minute on most computers.  

In [12]:
import time
start = time.time()
rf_grid_search =None
# rf_grid_search.fit(None, None)

# print("Testing Accuracy: {:.4}%".format(rf_grid_search.best_score_ * 100))
# print("Total Runtime for Grid Search on Random Forest Classifier: {:.4} seconds".format(time.time() - start))
# print("")
# print("Optimal Parameters: {}".format(rf_grid_search.best_params_))

### Interpreting Our Results

Did tuning the hyperparameters of our Random Forest Classifier improve model performance? Is this performance increase significant? Which model did better? If you had to choose, which model would you put into production? Explain your answer. 

Write your answer below this line:
________________________________________________________________________________________________________________________________


### Tuning Gradient Boosted Trees (AdaBoost)

The last model we'll tune in this lab is an AdaBoost Classifier, although tuning this model will generally be similar to tuning other forms of Gradient Boosted Tree (GBT) models.  

In the cell below, create an AdaBoost Classifier Object.  Then, as we did with the previous two examples, fit the model using using Cross Validation to get a baseline testing accuracy so we can see how an untuned AdaBoost model performs on this task.  

In [13]:
adaboost_clf = None
adaboost_mean_cv_score = None

# print("Mean Cross Validation Score for AdaBoost: {:.4}%".format(adaboost_mean_cv_score * 100))

Great! Now, onto creating the parameter grid for AdaBoost.  

Complete the `adaboost_param_grid` dictionary by adding in the following key-value pairs:

|   Parameters  |      Values     |
|:-------------:|:---------------:|
|  n_estimators |  [50, 100, 250] |
| learning_rate | [1.0, 0.5, 0.1] |

In [14]:
adaboost_param_grid = {
    
}

Great.  Now, for the finale--use Grid Search to find optimal parameters for AdaBoost, and see how the model performs overall!

In [15]:
adaboost_grid_search = None
# adaboost_grid_search.fit(None, None)

# print("Testing Accuracy: {:.4}%".format(adaboost_grid_search.best_score_ * 100))
# print("Total Runtime for Grid Search on AdaBoost: {:.4} seconds".format(time.time() - start))
# print("")
# print("Optimal Parameters: {}".format(adaboost_grid_search.best_params_))

## Summary

In this lab, we learned:

* How to iteratively search for optimal model parameters using `GridSearhCV`
* How to tune model parameters for Decision Trees, Random Forests, and AdaBoost models. 